In [1]:
import numpy as np
import time
import pandas as pd
import requests
from lxml import etree
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Crawling the Lianjia for house info

def retrieve(lis):
    if len(lis) > 0:
        return lis[0]
    else:
        return None


page = 30
headers = {'user-agent': 'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0;'}
res_lis = []

for p in tqdm(range(page)):
    url = 'https://sz.lianjia.com/ershoufang/pg%s'%p
    r = requests.get(url=url, verify=False, headers=headers)
    f = etree.HTML(r.text)
    for i in f.xpath("//div[@class='info clear']"):
        try:
            dic = {}
            content = i.xpath("./div")
            dic['title'] = retrieve(content[0].xpath("./a/text()"))
            dic['housecode'] = retrieve(content[0].xpath("./a/@data-housecode"))
            dic['link'] = retrieve(content[0].xpath("./a/@href"))

            dic['quater'] = retrieve(content[1].xpath(".//a/text()"))
            dic['quater_link'] = retrieve(content[1].xpath(".//a/@href"))
            line = [ii.strip() for ii in content[1].xpath("./div/text()")[0].split('|')]
            dic['layourt'] = line[1]
            dic['area'] = line[2]
            dic['orientation'] = line[3]
            dic['decoration'] = line[4]
            #dic['has_elevator'] = line[5]

            dic['floor'] = retrieve(content[2].xpath("./div/text()"))[:-5]
            dic['zone'] = retrieve(content[2].xpath(".//a/text()"))

            line = [ii.strip() for ii in content[3].xpath("text()")[0].split('/')]
            dic['follow'] = line[0]
            #dic['watched'] = line[1]
            dic['list_date'] = line[1]

            dic['subway'] =  retrieve(content[4].xpath("./span[@class='subway']/text()"))
            dic['tax_free'] =  retrieve(content[4].xpath("./span[@class='taxfree']/text()"))
            dic['vr'] =  retrieve(content[4].xpath("./span[@class='vr']/text()"))
            dic['has_key'] =  retrieve(content[4].xpath("./span[@class='haskey']/text()"))
            if len(content[4].xpath("./span[@class='good']")) > 0:
                dic['must_see'] = '必看好房'

            dic['price'] = retrieve(content[5].xpath("./div[@class='totalPrice']/span/text()")
                                   ) + retrieve(content[5].xpath("./div[@class='totalPrice']/text()"))
            dic['unit_price'] = retrieve(content[5].xpath("./div[@class='unitPrice']/@data-price"))

            res_lis.append(dic)

        except: pass
    time.sleep(0.5)

df = pd.DataFrame(res_lis)